<hr style="border-top-width:4px; border-top-color: #2188bf;">
<img align="right" src="images/Logo_Università_Padova.png" alt="Drawing" style="width: 300px"/>
<img align="right" src="images/Logo_Physics_of_Data.jpg" alt="Drawing" style="width: 135px"/>

# MAPD (mod.B) Final Project
# Analysis of Covid-19 papers

### Authors
* [Ardino Rocco](https://github.com/RoccoA97) (University of Padua)
* [Pagano Alice](https://github.com/AlicePagano) (University of Padua)
* [Valente Alessandro](https://github.com/mastrovalentz) (University of Padua)

<hr style="border-top-width:4px; border-top-color: #2188bf;">

<br>
<br>
<br>
<br>
<br>

<hr style="border-top-width:4px; border-top-color: #2188bf;">

# Index <a class="anchor" id="MAPD_B_FP.i"></a>

* 1. [**Introduction**](#MAPD_B_FP.1.)
* 2. [**Word counter distributed algorithm**](#MAPD_B_FP.2.)

<hr style="border-top-width:4px; border-top-color: #2188bf;">

<br>
<br>
<br>
<br>
<br>

<hr style="border-top-width:4px; border-top-color: #2188bf;">

## 1. - Introduction <a class="anchor" id="MAPD_B_FP.1."></a>
* [Index](#MAPD_B_FP.i)

In [1]:
from dask.distributed import Client
from dask import delayed

client = Client()
client

Client Scheduler: tcp://127.0.0.1:43451 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.80 GB


In [2]:
import dask.bag as db
import os
import json
from operator import itemgetter
from operator import add

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

from collections import Counter


In [3]:
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
js = lines.map(json.loads).repartition(10)

In [4]:
body = js.pluck("body_text")

In [5]:
#
#
#tokenizer = RegexpTokenizer(r'\w+')
#stop_words = set(stopwords.words('english'))
#
#
def merge(record):
    text=''
    for rec in record:
        text+=rec['text']
    return text

#texts = js.pluck("body_text").map(merge)
#texts.take(2, npartitions=2)
#
#def clean_func(text):
#
#    text_tokens = word_tokenize(text)
#    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
#
#    return tokens_without_sw

In [6]:

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

tokenizer = RegexpTokenizer(r'\w+')

no_words={"i", "as", "or", "it", "et", "also", "may"}
def clean_func(text):

    def merge_text(text,stop_words):
        new_sentence =''
        for w in text:
            if w.lower() not in stop_words and w.isalpha(): 
                new_sentence += w 
                new_sentence += " "
        return new_sentence

    stop_words = set(stopwords.words('english'))
    result = word_tokenize(text)
    result = merge_text(result,stop_words.union(no_words))
    result = tokenizer.tokenize(result)
    return result

texts = js.pluck("body_text").map(merge)
text_clean = texts.map(clean_func)


In [7]:

def count_words(text):
    counts = dict(zip(Counter(text).keys(), Counter(text).values()))
    wList = [{"word":x , "counts": y} for x,y in counts.items()]
    return wList

def order_words(text):
    return {k: v for k, v in sorted(text.items(), key=lambda item: item[1], reverse=True)}

words = text_clean.map(count_words)
#words_ordered = words.map(order_words)


words.take(1)

([{'word': 'Plasmodium', 'counts': 5},
  {'word': 'falciparum', 'counts': 27},
  {'word': 'causes', 'counts': 2},
  {'word': 'malignant', 'counts': 1},
  {'word': 'malaria', 'counts': 6},
  {'word': 'cerebral', 'counts': 1},
  {'word': 'CM', 'counts': 1},
  {'word': 'PAM', 'counts': 1},
  {'word': 'cause', 'counts': 1},
  {'word': 'death', 'counts': 1},
  {'word': 'individuals', 'counts': 1},
  {'word': 'annually', 'counts': 1},
  {'word': 'erythrocyte', 'counts': 1},
  {'word': 'membrane', 'counts': 1},
  {'word': 'protein', 'counts': 19},
  {'word': 'encoded', 'counts': 1},
  {'word': 'var', 'counts': 58},
  {'word': 'gene', 'counts': 36},
  {'word': 'family', 'counts': 6},
  {'word': 'comprising', 'counts': 2},
  {'word': 'approximately', 'counts': 2},
  {'word': 'members', 'counts': 4},
  {'word': 'major', 'counts': 5},
  {'word': 'virulence', 'counts': 3},
  {'word': 'factor', 'counts': 2},
  {'word': 'involved', 'counts': 9},
  {'word': 'antigenic', 'counts': 3},
  {'word': 'vari

In [ ]:
#def incr(tot, _):
#    return tot+1
#
#result = words.flatten().foldby(key='word', 
#                   binop=incr, 
#                   initial=0, 
#                   combine=add, 
#                   combine_initial=0).compute()
##print(sorted(result))
#result

In [8]:
#def amount_sum (x):
#    return np.sum(np.array([y['amount'] for y in x['transactions']]))
#
def incr_amount(tot, x):
    return tot+x['counts']

#def incr_amount(tot, x):
#    return tot+1

total_counts = words.flatten().foldby('word', binop=incr_amount, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()

total_counts

[('Plasmodium', 57),
 ('falciparum', 95),
 ('causes', 728),
 ('malignant', 171),
 ('malaria', 205),
 ('cerebral', 398),
 ('CM', 12),
 ('PAM', 14),
 ('cause', 1356),
 ('death', 1077),
 ('individuals', 1341),
 ('annually', 67),
 ('erythrocyte', 34),
 ('membrane', 1146),
 ('protein', 5049),
 ('encoded', 278),
 ('var', 124),
 ('gene', 2904),
 ('family', 1048),
 ('comprising', 113),
 ('approximately', 705),
 ('members', 658),
 ('major', 1578),
 ('virulence', 259),
 ('factor', 1316),
 ('involved', 1401),
 ('antigenic', 291),
 ('variation', 433),
 ('clinical', 4288),
 ('pathogenicity', 101),
 ('remarkable', 110),
 ('expression', 3567),
 ('mutually', 41),
 ('exclusive', 64),
 ('parasite', 291),
 ('time', 4485),
 ('one', 4710),
 ('genes', 2202),
 ('expressed', 1212),
 ('remaining', 376),
 ('silenced', 12),
 ('generation', 413),
 ('able', 1008),
 ('express', 346),
 ('different', 4573),
 ('switches', 27),
 ('lead', 853),
 ('results', 4390),
 ('immune', 2303),
 ('evasion', 50),
 ('chronic', 1073),

In [9]:
total_counts_ordered = sorted(total_counts, key=itemgetter(1), reverse=True)
total_counts_ordered

[('patients', 11586),
 ('cells', 10922),
 ('virus', 8326),
 ('study', 7376),
 ('infection', 7301),
 ('used', 7056),
 ('using', 6987),
 ('viral', 5890),
 ('data', 5870),
 ('disease', 5763),
 ('al', 5622),
 ('cell', 5244),
 ('two', 5076),
 ('protein', 5049),
 ('Fig', 4747),
 ('one', 4710),
 ('different', 4573),
 ('time', 4485),
 ('studies', 4426),
 ('results', 4390),
 ('clinical', 4288),
 ('could', 4074),
 ('number', 4047),
 ('human', 4018),
 ('cases', 4015),
 ('analysis', 3762),
 ('However', 3759),
 ('use', 3741),
 ('treatment', 3726),
 ('found', 3687),
 ('group', 3684),
 ('high', 3664),
 ('RNA', 3642),
 ('viruses', 3638),
 ('expression', 3567),
 ('respiratory', 3498),
 ('Figure', 3495),
 ('model', 3488),
 ('including', 3431),
 ('shown', 3392),
 ('associated', 3359),
 ('reported', 3332),
 ('control', 3323),
 ('first', 3279),
 ('system', 3190),
 ('response', 3168),
 ('well', 3123),
 ('proteins', 3061),
 ('infections', 2983),
 ('compared', 2961),
 ('Table', 2939),
 ('performed', 2908),
 (

In [ ]:
%%time


In [ ]:
def to_counter(record):
    return Counter(record)

words_counter= words_ordered.map(to_counter).reduction(Counter.update, sum).compute()
words_counter

In [ ]:
print(words_counter)

In [ ]:
w = words_ordered.take(1)[0]
from collections import Counter

z = dict(Counter(w)+Counter(w))

print(z)

In [ ]:
print(w.keys())

In [ ]:
words_ordered.groupby(lambda item: item.keys()).starmap(lambda k, v: (k, len(v))).compute()


In [ ]:
word= words.take(1)[0]
words_sorted = {k: v for k, v in sorted(word.items(), key=lambda item: item[1], reverse=True)}
words_sorted

In [ ]:
words.take(1)

In [ ]:
text_clean.take(2, npartitions=2)

In [ ]:
dataBag = db.read_text("data/papers_in_json_singleline/*.json").map(json.loads)

In [ ]:
dataBag.take(10000)

In [ ]:
! ls data/papers_in_json_singleline/*.json > converted_json.txt

In [ ]:
with open("converted_json.txt") as infile:
    content= readlines(infile)

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

3


stop = set(stopwords.words('english'))

text1 = texts.take(1)[0]
text_tokens = word_tokenize(text)

print(text_tokens)

text_clean = [t for t in text_tokens if t not in stop and t not in string.punctuation]
print(string.punctuation)


In [ ]:
print(string.punctuation)

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
nltk.download('punkt')

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

with open('moby.txt', 'r') as f:
    moby_raw = f.read()
    stop = set(stopwords.words('english'))
    moby_tokens = nltk.word_tokenize(moby_raw)
    text_no_stop_words_punct = [t for t in moby_tokens if t not in stop or t not in string.punctuation]

    print(text_no_stop_words_punct)